# Reliability

In earlier modules, we chose $F$ as the "frequency" variable and supposed that the average wait time was $1/2F$. But in real life, while $2F$ might be the frequency at which we send buses out of the depot, it's not necessarily the *frequency that passengers observe.* Anyone who has ever taken a bus more than a few times has probably had the following experience: You're waiting for a bus that's suppose to come at 5:45. The bus rolls up at 6:02 and, lo and behold, there's another bus right behind it! What happened? Are the drivers incompetent? Is it just bad luck?

It turns out there is a rigorous, systematic reason that buses bunch: in the language of dynamic systems, a bus system running at even headways is in an *unstable equilibrium*. That's a fancy way to say that even headways are fragile; the slightest shock to the system—a couple of wheelchair-bound passengers getting on at the same stop, for instance—will throw the buses out of sync. And once they're out of sync, the buses get trapped in a vicious cycle that even good drivers are powerless to get out of.

## The importance of even headways

Before we start looking at *why* buses bunch, we'll focus on understanding *why it matters.* The answer is that, ceteris paribus, more even headways mean lower aggregate waiting time. The figures below track the number of passengers waiting at a stop over time. Passengers arrive at the stop at a steady rate (the slope of the curves), and when a bus arrives the cumulative count drops to zero. The area under the curves is the amount of time spent waiting. The second figure, which shows uneven headways, has more area under the curve. So if we could stabilize headways we would not only meet our posted schedules but reduce the time spent waiting as well.

<img src="./img/wait-times.png" width="40%"/>


## Dynamical systems

To analyze bus bunching, we have to get familiar with the idea of **dynamical systems**. [Why they're not called "dynamic systems" is beyond me. Who gave "dynamic" the "al" on the end?] Dynamical systems have a "state" that evolves in time according to a set of rules. Before getting to bus bunching we'll look at some simpler systems.

### The parking lot
Consider a parking lot where $\lambda$ cars arrive every hour. A fraction $\rho$ of all cars parked at the start of every hour leave by the end of the hour. Let $x(t)$ be the number of cars in the lot at time $t$ (the state). In this case, the "motion" of the system is described by the equation

$$
\begin{align}
x(t+1) = x(t) + \lambda - \rho\cdot x(t)\\
x(t+1) = x(t)\cdot (1-\rho) + \lambda.\\
\end{align}
$$

Now, one way to analyze the system is to plot this equation, as below. The dark line is the equation for the state evolution. The blue dashed line is the identity line. Now, one thing you notice is that they intersect. That intersection occurs at a value $x_{eq}$ such that the number of cars stays the same from one period to the next. Thus, $x_{eq}$ is an **equilibrium** of the system. In this case it's the *only* one. Confirm for yourself that at values below $x_{eq}$, the number of cars in the lot will steadily rise; and for $x>x_{eq}$, the number of cars will decrease. (You can trace the path of the system from any point by drawing these kind of staircases as in the picture.) Therefore, $x_{eq}$ is a *stable equilibrium*: if you started at $x(t)=x_{eq}$ and slightly increased or decrease the number of cars, then the system would return to $x_{eq}$.

<img src="./img/dynamical.svg" width="40%"/>


We can also find $x_{eq}$ algebraically by setting:

$$
\begin{align}
x_{eq}&=\lambda + (1-\rho)\cdot x_{eq}\\
x_{eq} &= \frac{\lambda}{\rho}.
\end{align}
$$

And we can test the stability of the system by checking how the state responds to a slight perturbation. Suppose that we have a perturbation (a deviation from equilibrium)

$$
\varepsilon(t) = x(t)-x_{eq}.
$$

The question we ask about stability is: will this perturbation grow in time or decrease? To find out, we subtract 
$$
x_{eq} = \lambda + (1-\rho)x(t)
$$
from
$$
x(t+1) = \lambda + (1-\rho)x(t)
$$
to get
$$
\varepsilon(t+1)=(1-\rho)\varepsilon(t).
$$

Since $(1-\rho)<1$, it follows that $\varepsilon(t)$ declines as time goes on. Hence, $x_eq$ is a *stable* equilibrium.

### Unstable server system

Now imagine a server system where the number of people waiting reduces the efficiency of the server. For example, consider a yogurt counter in a country where there isn't a culture of standing in lines; everyone just crowds up front. If too many people are crowded around the register yelling their order, the guy at the register can't hear the orders. Suppose that customers arrive at rate $\lambda$, and the counter can process $\gamma - \rho x(t)$ people per minute, where $x(t)$ is the number of people yelling their order and $\gamma>\lambda$. The state equation is

$$
x(t+1) = \max\{0,x(t)+\lambda - (\gamma - \rho x(t)) \} = [\lambda - \gamma + (1+\rho) x(t)]^+,
$$
where we need the $[]^+$ because it's impossible to have fewer than 0 people waiting.


Plotting this equation like we did before gives. This time, the state equation line intersects $x(t+1)=x(t)$ from below. So if $x(t)<x_eq$, then $x(t)$ will keep declining all the way down to zero; and if $x(t)>x_eq$ then the line will keep getting longer and longer.

<img src="./img/unstable-server.svg" width="50%"/>


Again, we can look at things algebraically by subtracting $x(t+1)-x_eq = \varepsilon(t+1)$, which gives

$$
\varepsilon(t+1) = (1+\rho)\varepsilon(t).
$$

Since $(1+\rho) > 1$, it follows that perturbations keep growing.


## A system of buses

Now we are going to apply these tools to a system of buses. Buses have to travel between stops, where they pick people up. Instability arises from two facts: (i) people take time to board the bus; (ii) the later the bus is, the more people will be waiting at the stop. So if a bus gets behind, then the extra people will slow it down even further.


### Uncontrolled bus system

First we are going to look at a system with no control at all. Our model has these parameters:


+ $t_{n,s}$ the *scheduled* time that bus $n$ arrives at stop $s$
+ $a_{n,s}$ the *actual* time that bus $n$ arrives at stop $s$
+ $\beta$ the boarding time per passenger (min)
+ $\lambda_s$ the arrival rate of passengers at stop $s$ (pax/min)
+ $c_s$ the time required to travel from stop $s$ to $s+1$

Once a bus $n$ pulls into stop $s$, it needs to pick up everyone waiting there. The amount of time since the last arrival is, by definition, $t_{n,s}-t_{n-1,s}$. Therefore, the number of people waiting is $\lambda_s(t_{n,s}-t_{n-1,s})$, and so the bus has to spend $\beta \lambda_s (t_{n,s}-t_{n-1,s})$ at the stop. It follows that the following equation holds for the *scheduled* arrival time at stop $s+1$:

$$
t_{n,s+1} = t_{n,s} + c_s + \lambda_s \beta\cdot (t_{n,s} - t_{n-1,s}),
$$

where the first term is the arrival time at the last stop, the second term how long it takes to get to the next stop and the last term the amount of time spent at the last stop. This equation will serve the same purpose as the equilibrium equation in the previous models.

Now, we are interesting in knowing what happens if a single bus, however briefly, gets off schedule. Suppose bus $n$ gets slightly off schedule at stop $s$. What happens at stop $s+1$? Well, given that bus $n-1$ was on time at stop $s$, the following equation holds:

$$
a_{n,s+1} = a_{n,s} + c_s + \lambda_s \beta\cdot (a_{n,s} - t_{n-1,s}),
$$

where $a_{n,s}$ is the actual (not scheduled time $n$ gets to $s$. 

Now, lets define 
$$
\varepsilon_{n,s} = a_{n,s} - t_{n,s}
$$
to be bus $n$'s deviation from schedule at stop $s$. Likewise, $\varepsilon_{n,s+1} = a_{n,s+1} - t_{n,s+1}$ is the same figure for stop $s+1$. To see how $\varepsilon_{n,s+1}$ depends on $\varepsilon_{n,s}$, we subtract (term by term)

$$
\begin{align}
\varepsilon_{n,s+1} & = a_{n,s+1} - t_{n,s+1} = (a_{n,s} - t_{n,s}) + (c_s-c_s) + \lambda_s \beta \cdot \bigg((a_{n,s} - t_{n,s}) - (t_{n-1,s} - t_{n-1}) \bigg)\\
\varepsilon_{n,s+1} &= \varepsilon_{n,s} + 0 + \beta \lambda_s \cdot \bigg( \varepsilon_{n,s} - 0 \bigg) \\
\varepsilon_{n,s+1} &= \varepsilon_{n,s}\cdot(1+\beta \lambda_s)
\end{align}
$$

So apparently this system is more like the (unstable) counter system than the (stable) parking lot system. A small perturbation grows and grows. If the bus is late at stop $s$, it will be even later at stop $s+1$. And if the bus is early at stop $s$, it will be even earlier at stop $s+1$. 

Here is the analysis plot for this situation:

INSRT PLOT

### Bus with waiting

Now we are going to look at a minor version of schedule control: suppose that if a bus would arrive early at stop $s+1$, then it pauses somewhere along the way or waits until it's time to leave at the previous stop. This ensures that an early arrival at stop $s$ doesn't lead to an early arrival at $s+1$. In that case, we have the equation

$$
a_{n,s+1} = \max \bigg\{ t_{n,s+1},a_{n,s} + c_s + \lambda_s \beta\cdot (a_{n,s} - t_{n-1,s})\bigg\}.
$$

If we subtract $t_{n,s+1}$ from this expression we get
$$
\varepsilon_{n,s+1} = \bigg[\varepsilon_{n,s}\cdot(1+\beta \lambda_s) \bigg]^+,
$$
where $[x]^+ := \max\{0,x\}$ is a function that ensures $x$ isn't negative. Once we include this light control, our state equation is truncated at the origin. Being early isn't a problem anymore.

### Bus with slack

The above waiting strategy eliminated deviations caused by earliness. But what about lateness? One way to deal with lateness is to incorporate **slack** into the schedule. This is how real agencies try to combat bus bunching.

Slack works by adding to the schedule a time $\phi_s$ beyond what's necessary to drive between $s$ and $s+1$. Maybe the bus stops between two stops for a little bit, or holds extra time at a stop. With slack, the schedule equation is 

$$
t_{n,s+1} = t_{n,s} + c_s  + \phi_s + \lambda_s \beta\cdot (t_{n,s} - t_{n-1,s}).
$$

Now, obviously, a bus doesn't have to use all of its slack.  Therefore, we have
$$
a_{n,s+1} = \max\bigg\{t_{n,s+1}, a_{n,s} + c_s + \lambda_s \beta\cdot (a_{n,s} - t_{n-1,s}) \bigg\}.
$$

What this equation says is: If the bus is going to be late at $s+1,$ then it skips the slack (then you use the second term in brackets. Otherwise, it uses whatever amount of slack needed to arrive on time at $s+1$, where "on time" has been defined to include $\phi_s$ worth of slack.

If we subtracted $t_{n,s+1}$ from $a_{n,s+1}$ now, we get

$$
\varepsilon_{n,s+1} = \bigg[\varepsilon_{n,s}\cdot(1+\beta \lambda_s) - \phi_s \bigg]^+.
$$

INSERT PLOT